In [21]:
import pandas as pd

In [22]:
df=pd.read_csv('datos/datos_filtrados_sin_encoding.csv')

In [25]:
len(df)

7352

In [27]:
max([len(i) for i in df['original_sequence']])

1597

In [3]:
display(df[df['is_training']==True]['genus'].value_counts())
len(df[df['is_training']==True])

genus
Flavobacterium       120
Bifidobacterium      120
Rhizobium            120
Microbacterium       120
Mesorhizobium        120
Acinetobacter        120
Collinsella          120
Pelagibacter_A       120
Arthrobacter         120
Nocardia             120
Micromonospora       120
Vibrio               120
Chryseobacterium     120
Mycobacterium        120
Nocardioides         120
Corynebacterium      120
Pseudomonas_E        120
Pelagibacter         120
Bradyrhizobium       120
Sphingomonas         120
Streptomyces         120
Prochlorococcus_A    120
Streptococcus        120
Paraburkholderia     118
Polynucleobacter     113
Pedobacter           112
Prevotella           105
Paracoccus           103
Novosphingobium      102
Methylobacterium     101
Name: count, dtype: int64

3514

In [4]:
display(df[df['is_training']==False]['genus'].value_counts())
len(df[df['is_training']==False])

genus
Streptomyces         802
Pelagibacter         657
Pseudomonas_E        560
Streptococcus        343
Mycobacterium        257
Flavobacterium       245
Microbacterium       162
Prochlorococcus_A    113
Bradyrhizobium       109
Sphingomonas         104
Corynebacterium       81
Vibrio                63
Arthrobacter          60
Chryseobacterium      52
Acinetobacter         50
Nocardioides          49
Rhizobium             29
Collinsella           28
Micromonospora        23
Mesorhizobium         22
Nocardia              17
Bifidobacterium        8
Pelagibacter_A         4
Name: count, dtype: int64

3838

In [5]:
display(df['genus'].value_counts())
len(df)

genus
Streptomyces         922
Pelagibacter         777
Pseudomonas_E        680
Streptococcus        463
Mycobacterium        377
Flavobacterium       365
Microbacterium       282
Prochlorococcus_A    233
Bradyrhizobium       229
Sphingomonas         224
Corynebacterium      201
Vibrio               183
Arthrobacter         180
Chryseobacterium     172
Acinetobacter        170
Nocardioides         169
Rhizobium            149
Collinsella          148
Micromonospora       143
Mesorhizobium        142
Nocardia             137
Bifidobacterium      128
Pelagibacter_A       124
Paraburkholderia     118
Polynucleobacter     113
Pedobacter           112
Prevotella           105
Paracoccus           103
Novosphingobium      102
Methylobacterium     101
Name: count, dtype: int64

7352

In [ ]:
genes = df['genus'].unique()

# Configuración de datos por clase
MIN_DATA_PER_CLASS = 200         # Mínimo de secuencias para entrenamiento
MAX_DATA_PER_CLASS = 200         # Máximo de secuencias para entrenamiento  
MAX_TOTAL_PER_CLASS = 500        # Máximo total de secuencias (training + test)

# Filtro para los géneros que tienen más de 100 representantes
# Separa datos entre entrenamiento y testeo basado en MIN/MAX configurado
templist = []

for i in genes:
    temp = df[df['genus']==i]
    temp = temp.sort_values('sequence_length', ascending=False)
    
    total_sequences = len(temp)
    
    # Primero limitar el total de secuencias si excede MAX_TOTAL_PER_CLASS
    if total_sequences > MAX_TOTAL_PER_CLASS:
        temp = temp.head(MAX_TOTAL_PER_CLASS)
        total_sequences = MAX_TOTAL_PER_CLASS
    
    # Determinar cuántas secuencias usar para entrenamiento
    if MIN_DATA_PER_CLASS == MAX_DATA_PER_CLASS:
        # Si min == max, usar exactamente ese número para entrenamiento
        train_size = min(MIN_DATA_PER_CLASS, total_sequences)
    else:
        # Si min != max, aplicar lógica de rango
        if total_sequences >= MAX_DATA_PER_CLASS:
            train_size = MAX_DATA_PER_CLASS
        elif total_sequences >= MIN_DATA_PER_CLASS:
            train_size = total_sequences
        else:
            train_size = total_sequences
    
    # Calcular tamaño de test
    test_size = total_sequences - train_size
    
    # Marcar datos de entrenamiento y testeo
    temp['is_training'] = False
    temp['is_test'] = False
    
    # Asignar entrenamiento
    temp.iloc[:train_size, temp.columns.get_loc('is_training')] = True
    
    # Asignar test (si hay datos restantes)
    if test_size > 0:
        temp.iloc[train_size:train_size+test_size, temp.columns.get_loc('is_test')] = True
    
    # print(f"{i}: Total={total_sequences}, Train={train_size}, Test={test_size}")
    
    templist.append(temp)

# Concatenar todos los datos
df_final = pd.concat(templist)
df_final = df_final.reset_index(drop=True)

# Estadísticas finales
print(f"\nEstadísticas finales:")
print(f"Total secuencias: {len(df_final)}")
print(f"Entrenamiento: {df_final['is_training'].sum()}")
print(f"Test: {df_final['is_test'].sum()}")
print(f"Géneros procesados: {df_final['genus'].nunique()}")

# Guardar resultado
df_final.to_csv('datos/ddaug.csv', index=False)


Estadísticas finales:
Total secuencias: 14344
Entrenamiento: 5800
Test: 8544
Géneros procesados: 29


In [14]:
df_final['genus'].value_counts()

genus
Acinetobacter       500
Arthrobacter        500
Streptomyces        500
Streptococcus       500
Sphingomonas        500
Rhizobium           500
Pseudomonas         500
Prevotella          500
Polynucleobacter    500
Pelagibacter        500
Pedobacter          500
Paracoccus          500
Paraburkholderia    500
Novosphingobium     500
Nocardioides        500
Nocardia            500
Mycobacterium       500
Micromonospora      500
Microbacterium      500
Methylobacterium    500
Mesorhizobium       500
Flavobacterium      500
Corynebacterium     500
Chryseobacterium    500
Bradyrhizobium      500
Bifidobacterium     500
Vibrio              500
Prochlorococcus     428
Collinsella         416
Name: count, dtype: int64

In [18]:
df = df_final

# Cargar datos originales para calcular descartados
df_original = pd.read_csv('datos/datos_nuevos_juntos.csv')
original_counts = df_original['genus'].value_counts()

# Obtener conteos por grupo del dataset final
train_counts = df[df['is_training'] == True]['genus'].value_counts()
test_counts = df[df['is_test'] == True]['genus'].value_counts()
used_counts = df['genus'].value_counts()  # Total usado (train + test)

# Obtener totales generales
total_train = len(df[df['is_training'] == True])
total_test = len(df[df['is_test'] == True])
total_used = len(df)
total_genera = len(df['genus'].unique())

# Calcular totales descartados
total_original = sum(original_counts[genus] for genus in used_counts.index)
total_discarded = total_original - total_used

# Crear DataFrame combinado
combined_data = []
for genus in used_counts.index:
    train_count = train_counts.get(genus, 0)
    test_count = test_counts.get(genus, 0)
    used_count = used_counts[genus]
    original_count = original_counts.get(genus, used_count)  # Por si acaso no está en originales
    discarded_count = original_count - used_count
    
    combined_data.append({
        'genus': genus,
        'train': train_count,
        'test': test_count,
        'used': used_count,
        'discarded': discarded_count,
        'original': original_count
    })

# Convertir a DataFrame y ordenar por total original (descendente)
table_df = pd.DataFrame(combined_data)
table_df = table_df.sort_values('original', ascending=False)

# Obtener rango de secuencias usadas y longitudes
min_sequences = table_df['used'].min()
max_sequences = table_df['used'].max()

# Calcular rango de longitudes de secuencias
min_length = df['sequence_length'].min()
max_length = df['sequence_length'].max()

# Generar código LaTeX
latex_code = []
latex_code.append("\\begin{table}[htbp]")
latex_code.append("\\centering")
latex_code.append("\\caption{Distribución de secuencias por género bacteriano en el conjunto de datos final}")
latex_code.append("\\label{tab:genus_distribution}")
latex_code.append("\\setlength{\\tabcolsep}{6pt}")
latex_code.append("\\renewcommand{\\arraystretch}{1.2}")
latex_code.append("\\begin{tabular}{lrrrrr}")
latex_code.append("\\hline")
latex_code.append("\\textbf{Género} & \\textbf{Entrenamiento} & \\textbf{Test} & \\textbf{Usado} & \\textbf{Descartado} & \\textbf{Original} \\\\")
latex_code.append("\\hline")

# Agregar filas de datos
for _, row in table_df.iterrows():
    genus = row['genus'].replace('_', '\\_')  # Escapar guiones bajos para LaTeX
    train = row['train'] if row['train'] > 0 else '--'
    test = row['test'] if row['test'] > 0 else '--'
    used = row['used']
    discarded = row['discarded'] if row['discarded'] > 0 else '--'
    original = row['original']
    latex_code.append(f"{genus} & {train} & {test} & {used} & {discarded} & {original} \\\\")

# Agregar totales y estadísticas
latex_code.append("\\hline")
latex_code.append(f"\\textbf{{Total}} & \\textbf{{{total_train:,}}} & \\textbf{{{total_test:,}}} & \\textbf{{{total_used:,}}} & \\textbf{{{total_discarded:,}}} & \\textbf{{{total_original:,}}} \\\\")
latex_code.append("\\hline")
latex_code.append(f"\\multicolumn{{6}}{{l}}{{\\textbf{{Distribución: {total_genera} géneros bacterianos procesados}}}} \\\\")
latex_code.append(f"\\multicolumn{{6}}{{l}}{{\\textbf{{Rango utilizado: {min_sequences}-{max_sequences} secuencias por género}}}} \\\\")
latex_code.append(f"\\multicolumn{{6}}{{l}}{{\\textbf{{Longitud de secuencias: {min_length}-{max_length} pares de bases}}}} \\\\")
latex_code.append(f"\\multicolumn{{6}}{{l}}{{\\textbf{{Descartado: secuencias originales no incluidas en el dataset final}}}} \\\\")
latex_code.append("\\hline")
latex_code.append("\\end{tabular}")
latex_code.append("\\end{table}")

# Unir todas las líneas
full_latex = '\n'.join(latex_code)
    
print(full_latex)

# Mostrar estadísticas adicionales
print(f"\n=== ESTADÍSTICAS DEL PROCESAMIENTO ===")
print(f"Secuencias originales disponibles: {total_original:,}")
print(f"Secuencias utilizadas (train+test): {total_used:,}")
print(f"Secuencias descartadas: {total_discarded:,}")
print(f"Porcentaje de aprovechamiento: {(total_used/total_original)*100:.1f}%")
print(f"Distribución final: {total_train:,} entrenamiento + {total_test:,} test")

\begin{table}[htbp]
\centering
\caption{Distribución de secuencias por género bacteriano en el conjunto de datos final}
\label{tab:genus_distribution}
\setlength{\tabcolsep}{6pt}
\renewcommand{\arraystretch}{1.2}
\begin{tabular}{lrrrrr}
\hline
\textbf{Género} & \textbf{Entrenamiento} & \textbf{Test} & \textbf{Usado} & \textbf{Descartado} & \textbf{Original} \\
\hline
Streptomyces & 200 & 300 & 500 & 1355 & 1855 \\
Pseudomonas & 200 & 300 & 500 & 1094 & 1594 \\
Streptococcus & 200 & 300 & 500 & 901 & 1401 \\
Mycobacterium & 200 & 300 & 500 & 808 & 1308 \\
Flavobacterium & 200 & 300 & 500 & 744 & 1244 \\
Microbacterium & 200 & 300 & 500 & 728 & 1228 \\
Bradyrhizobium & 200 & 300 & 500 & 683 & 1183 \\
Sphingomonas & 200 & 300 & 500 & 640 & 1140 \\
Arthrobacter & 200 & 300 & 500 & 625 & 1125 \\
Rhizobium & 200 & 300 & 500 & 587 & 1087 \\
Mesorhizobium & 200 & 300 & 500 & 587 & 1087 \\
Micromonospora & 200 & 300 & 500 & 582 & 1082 \\
Acinetobacter & 200 & 300 & 500 & 564 & 1064 \\
Chryseoba

In [11]:
df_final[['index', 'sequence_id', 'sequence', 'sequence_length', 'gc_content',
       'domain', 'phylum', 'class', 'order', 'family', 'genus', 'species',
       'is_training']].to_csv("datos/datos_daug.csv", index=False)

In [7]:
df=df_final
# Obtener conteos por grupo
train_counts = df[df['is_training'] == True]['genus'].value_counts()
test_counts = df[df['is_training'] == False]['genus'].value_counts()
total_counts = df['genus'].value_counts()

# Obtener totales generales
total_train = len(df[df['is_training'] == True])
total_test = len(df[df['is_training'] == False])
total_general = len(df)
total_genera = len(df['genus'].unique())

# Crear DataFrame combinado
combined_data = []
for genus in total_counts.index:
    train_count = train_counts.get(genus, 0)
    test_count = test_counts.get(genus, 0)
    total_count = total_counts[genus]
    
    combined_data.append({
        'genus': genus,
        'train': train_count,
        'test': test_count,
        'total': total_count
    })

# Convertir a DataFrame y ordenar por total (descendente)
table_df = pd.DataFrame(combined_data)
table_df = table_df.sort_values('total', ascending=False)

# Obtener rango de secuencias
min_sequences = table_df['total'].min()
max_sequences = table_df['total'].max()

# Generar código LaTeX
latex_code = []
latex_code.append("\\begin{table}[htbp]")
latex_code.append("\\centering")
latex_code.append("\\caption{Distribución de secuencias por género bacteriano en el conjunto de datos final}")
latex_code.append("\\label{tab:genus_distribution}")
latex_code.append("\\setlength{\\tabcolsep}{8pt}")
latex_code.append("\\renewcommand{\\arraystretch}{1.2}")
latex_code.append("\\begin{tabular}{lrrr}")
latex_code.append("\\hline")
latex_code.append("\\textbf{Género} & \\textbf{Entrenamiento} & \\textbf{Test} & \\textbf{Total} \\\\")
latex_code.append("\\hline")

# Agregar filas de datos
for _, row in table_df.iterrows():
    genus = row['genus'].replace('_', '\\_')  # Escapar guiones bajos para LaTeX
    train = row['train'] if row['train'] > 0 else '--'
    test = row['test'] if row['test'] > 0 else '--'
    total = row['total']
    latex_code.append(f"{genus} & {train} & {test} & {total} \\\\")

# Agregar totales y estadísticas
latex_code.append("\\hline")
latex_code.append(f"\\textbf{{Total}} & \\textbf{{{total_train:,}}} & \\textbf{{{total_test:,}}} & \\textbf{{{total_general:,}}} \\\\")
latex_code.append("\\hline")
latex_code.append(f"\\multicolumn{{4}}{{l}}{{\\textbf{{Distribución: {total_genera} géneros bacterianos}}}} \\\\")
latex_code.append(f"\\multicolumn{{4}}{{l}}{{\\textbf{{Rango: {min_sequences}-{max_sequences} secuencias por género}}}} \\\\")
latex_code.append("\\hline")
latex_code.append("\\end{tabular}")
latex_code.append("\\end{table}")

# Unir todas las líneas
full_latex = '\n'.join(latex_code)
    
print(full_latex)

\begin{table}[htbp]
\centering
\caption{Distribución de secuencias por género bacteriano en el conjunto de datos final}
\label{tab:genus_distribution}
\setlength{\tabcolsep}{8pt}
\renewcommand{\arraystretch}{1.2}
\begin{tabular}{lrrr}
\hline
\textbf{Género} & \textbf{Entrenamiento} & \textbf{Test} & \textbf{Total} \\
\hline
Streptomyces & 200 & 1655 & 1855 \\
Pseudomonas & 200 & 1394 & 1594 \\
Streptococcus & 200 & 1201 & 1401 \\
Mycobacterium & 200 & 1108 & 1308 \\
Flavobacterium & 200 & 1044 & 1244 \\
Microbacterium & 200 & 1028 & 1228 \\
Bradyrhizobium & 200 & 983 & 1183 \\
Sphingomonas & 200 & 940 & 1140 \\
Arthrobacter & 200 & 925 & 1125 \\
Rhizobium & 200 & 887 & 1087 \\
Mesorhizobium & 200 & 887 & 1087 \\
Micromonospora & 200 & 882 & 1082 \\
Acinetobacter & 200 & 864 & 1064 \\
Chryseobacterium & 200 & 858 & 1058 \\
Corynebacterium & 200 & 858 & 1058 \\
Nocardioides & 200 & 858 & 1058 \\
Paracoccus & 200 & 847 & 1047 \\
Bifidobacterium & 200 & 843 & 1043 \\
Nocardia & 200 & 841 &

In [20]:
df_final.to_csv('datos/80m.csv', index=False)